# **Модуль sqlite3**
SQLite - это библиотека C, которая обеспечивает легкую базу данных на диске, не требующую отдельного серверного процесса и позволяющую получить доступ к базе данных с использованием нестандартного варианта языка запросов SQL. Некоторые приложения могут использовать SQLite для внутреннего хранения данных. Также можно создать прототип приложения с помощью SQLite, а затем перенести код в более крупную базу данных, такую ​​как PostgreSQL или Oracle.

## **Использование модуля**

Чтобы использовать модуль, нужно сначала создать объект Connection, представляющий базу данных. В примере данные будут храниться в файле Groups.db:

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')
conn.close()

Также можно указать специальное имя: memory: для создания базы данных в RAM. 

```
db = sqlite3.connect(":memory:")
```

Когда у вас есть Connection, можно создать объект Cursor и вызвать его метод execute() (выполнить) для выполнения команд SQL:

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')

c = conn.cursor()

# Создаем таблицу Groups
c.execute('''CREATE TABLE IF NOT EXISTS Groups
             (group_id INTEGER PRIMARY KEY ON CONFLICT REPLACE, group_name TEXT, group_course SMALLINT)''')

#Добавляем в нее первую запись
c.execute('''INSERT INTO Groups VALUES (1, "ИБ-1801", 3)''')

conn.commit() #Сохранить изменения

conn.close()

Обычно операциям SQL нужно использовать значения из переменных в Python. Не стоит делать запрос с помощью строковых операций Python, потому что это небезопасно; это делает вашу программу уязвимой для атаки SQL-инъекций (см. https://xkcd.com/327/ для юмористического примера того, что может пойти не так).

Вместо этого можно использовать подстановку параметров DB-API. Пишите ? в качестве параметра везде, где вы хотите использовать значение, а затем подставьте кортеж значений в качестве второго аргумента методу execute() курсора.
Например:

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')

c = conn.cursor()

c.execute('''INSERT INTO Groups VALUES (2, "ИБ-1802", 3)''') #добавим еще одну группу

conn.commit()

# Так никогда не делайте --это небезопасно!
name = 'ИБ-1801'
c.execute("SELECT * FROM Groups WHERE group_name = '%s'" % name)
print(f'Небезопасный запрос 1: {c.fetchone()}')

name = "' union SELECT * FROM Groups WHERE group_id = '2'  or '5'='5''"
c.execute("SELECT * FROM Groups WHERE group_name = '%s'" % name)
print(f'Небезопасный запрос 2: {c.fetchone()} - сработала инъекция с выводом элемента с group_id = 2') 

# Делайте так
name = ('ИБ-1802',)
c.execute("SELECT * FROM Groups WHERE group_name = ?", name)
print(f'Более безопасный вариант - {c.fetchone()}')

Небезопасный запрос 1: (3, 'ИБ-1801', 3)
Небезопасный запрос 2: (2, 'ИБ-1802', 3) - сработала инъекция с выводом элемента с group_id = 2
Более безопасный вариант - (2, 'ИБ-1802', 3)


Для добавления большего количества записей в БД можно использовать метод executemany объекта Connection

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')

c = conn.cursor()

# Пример с добавлением большего количества записей
gr = [
        (3, 'ИБ-1801', 3),
        (4, 'ПМ-1801', 3),
        (5, 'ПМ-1802', 3),
      ]
c.executemany('INSERT INTO Groups VALUES (?,?,?)', gr)
groups = c.execute("SELECT * FROM Groups")


for i in groups:
  print(i) 

conn.commit()

conn.close()

(1, 'БИ-1801', 3)
(2, 'ИБ-1802', 3)
(3, 'ИБ-1801', 3)
(4, 'ПМ-1801', 3)
(5, 'ПМ-1802', 3)


Чтобы получить данные после выполнения инструкции SELECT, вы можете рассматривать курсор как итератор (как выше в выводе), вызвать метод курсора fetchone() для получения единственной совпадающей строки или вызвать функцию fetchall() для получения списка совпадающих строк:

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')

c = conn.cursor()

c.execute("SELECT * FROM Groups")
c.fetchall() 

# c.close()

## **Основные функции и константы модуля sqlite3**

### sqlite3.**version**

Номер версии этого модуля в виде строки. Это не версия библиотеки SQLite.

### sqlite3.**version_info**

Номер версии этого модуля, как кортеж целых чисел. Это не версия библиотеки SQLite.

### sqlite3.**sqlite_version**

Номер версии библиотеки SQLite ***во время выполнения*** в виде строки.

### sqlite3.**sqlite_version_info**

Номер версии библиотеки SQLite ***во время выполнения*** в виде кортежа целых чисел.



In [ ]:
import sqlite3

print(f'{sqlite3.version}, {sqlite3.version_info}, {sqlite3.sqlite_version}, {sqlite3.sqlite_version_info}')

2.6.0, (2, 6, 0), 3.22.0, (3, 22, 0)


### **sqlite3.connect**(database[, timeout, detect_types, isolation_level, check_same_thread, factory, cached_statements, uri])

Подключение к БД.

**detect_types** по умолчанию равен 0 (т.е. выкл., без определения типа), вы можете установить для него любую комбинацию:

PARSE_DECLTYPES - При установке этого параметра модуль sqlite3 анализирует объявленный тип для каждого возвращаемого столбца 

PARSE_COLNAMES - При установке этого параметра SQLite анализирует имя каждого возвращаемого столбца

Если **uri** = True, база данных интерпретируется как URI. Это позволяет указать параметры. Например, чтобы открыть базу данных в режиме только для чтения, вы можете использовать: 

`db = sqlite3.connect('file:path/to/database?mode=ro', uri=True)`

[URI](https://ru.wikipedia.org/wiki/URI) — символьная строка, позволяющая идентифицировать какой-либо ресурс: документ, изображение, файл, службу, ящик электронной почты и т. д. 

### sqlite3.**complete_statement**(sql)

Возвращает True, если строка sql содержит один или несколько полных операторов SQL, заканчивающихся точкой с запятой. Не проверяет синтаксическую корректность SQL, а только то, что в нем нет незамкнутых строковых литералов и оператор завершается точкой с запятой.

Пример использования:

In [ ]:
import sqlite3


print(sqlite3.complete_statement('SELECT 10 > 5;'))
print(sqlite3.complete_statement('SELECT 10 > 5;"'))
print(sqlite3.complete_statement('SELECT "10" > "5";'))



True
False
True


## **Объекты Connection**

Класс sqlite3.Connection

Подключение к базе данных SQLite имеет следующие основные атрибуты и методы:

### **commit()**
Этот метод фиксирует текущее изменение. Если вы не вызвали этот метод, все, что вы делали с момента последнего вызова commit(), не будет видно из других подключений к базе данных.

### **rollback()**

Этот метод откатывает любые изменения в базе данных с момента последнего вызова commit().

### **close()**
Закрывает соединение с базой данных. Обратите внимание, что он автоматически не вызывает commit(). Если вы просто закроете соединение с базой данных без предварительного вызова commit(), ваши изменения будут потеряны!

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')

c = conn.cursor()

#Добавляем в нее первую запись
c.execute('''INSERT INTO Groups VALUES (1, "БИ-1801", 3)''')

conn.commit() #Сохранить изменения

c.execute('''INSERT INTO Groups VALUES (1, "ИБ-1803", 3)''')

c.execute('''INSERT INTO Groups VALUES (1, "ИБ-1803", 3)''')

conn.rollback()

c.execute("SELECT * FROM Groups")
c.fetchall() 


# conn.close()

[(1, 'БИ-1801', 3),
 (2, 'ИБ-1802', 3),
 (3, 'ИБ-1801', 3),
 (4, 'ПМ-1801', 3),
 (5, 'ПМ-1802', 3)]

## **Объекты Cursor**
Класс sqlite3.Cursor

Экземпляр Cursor имеет следующие атрибуты и методы:

### **execute**(sql[, параметры])
Выполняет инструкцию SQL. Оператор SQL может быть параметризован (т. е. заполнители вместо литералов SQL). Модуль sqlite3 поддерживает два вида заполнителей: вопросительные знаки (стиль qmark) и именованные заполнители (именованный стиль).

Вот пример обоих стилей:

In [ ]:
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table people (name, age)")

who = "Rakhman"
age = 19

# This is the qmark style:
cur.execute("insert into people values (?, ?)", (who, age))

# And this is the named style:
cur.execute("select * from people where name=:who and age=:age", {"who": who, "age": age})

print(cur.fetchone())

con.close()

('Rakhman', 19)


### **executemany**(sql, seq_of_parameters)
Выполняет команду SQL для всех последовательностей параметров или отображений, найденных в последовательности seq_of_parameters. Модуль sqlite3 также позволяет использовать итератор, выдающий параметры вместо последовательности.

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')

c = conn.cursor()

# Пример с добавлением большего количества записей
gr = [
        (3, 'ИБ-1801', 3),
        (4, 'ПМ-1801', 3),
        (5, 'ПМ-1802', 3),
      ]
c.executemany('INSERT INTO Groups VALUES (?,?,?)', gr)
groups = c.execute("SELECT * FROM Groups")


for i in groups:
  print(i) 

conn.commit()

conn.close()

(1, 'БИ-1801', 3)
(2, 'ИБ-1802', 3)
(3, 'ИБ-1801', 3)
(4, 'ПМ-1801', 3)
(5, 'ПМ-1802', 3)


### **executescript**(sql, seq_of_parameters)
Это нестандартный удобный метод для выполнения нескольких функций SQL одновременно. Сначала он использует инструкцию COMMIT, а затем выполняет SQL-скрипт, который получает в качестве параметра.

sql_script может быть экземпляром str.

In [ ]:
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.executescript("""
    create table person(
        firstname,
        lastname,
        age
    );

    create table book(
        title,
        author,
        published
    );

    insert into book(title, author, published)
    values (
        'Dirk Gently''s Holistic Detective Agency',
        'Douglas Adams',
        1987
    );
    """)
con.close()

### **fetchone()**
Выбирает строку запроса, возвращая одну строку или None, когда нет доступных для возврата данных.

### **fetchmany**(size=cursor.arraysize)
Выбирает набор строк результата запроса, возвращая list.
Количество строк для выборки за один вызов определяется параметром size.

Обратите внимание, что с параметром size связаны некоторые соображения производительности. Для оптимальной производительности лучше использовать атрибут arraysize. Если используется параметр size, то лучше всего, чтобы он сохранял одно и то же значение от одного вызова fetchmany() до следующего

### **fetchall()**
Выбирает все строки результата запроса, возвращая list.

In [ ]:
import sqlite3

conn = sqlite3.connect('Groups.db')

c = conn.cursor()

c.execute("SELECT * FROM Groups")

print(c.fetchmany(3))
c.execute("SELECT * FROM Groups WHERE group_name LIKE 'ПМ%'") 
print(c.fetchall())
# print(c.fetchone())

[(1, 'БИ-1801', 3), (2, 'ИБ-1802', 3), (3, 'ИБ-1801', 3)]
[(4, 'ПМ-1801', 3), (5, 'ПМ-1802', 3)]


### **lastrowid**

Этот доступный только для чтения атрибут предоставляет идентификатор последней измененной строки. Он устанавливается, только если вы выполнили оператор INSERT или REPLACE с помощью метода execute(). Для операций, отличных от INSERT или REPLACE, или когда вызывается executemany (), lastrowid устанавливается в None.

In [ ]:
import sqlite3

conn = sqlite3.connect('Groups.db')

c = conn.cursor()

c.execute('''INSERT INTO Groups VALUES (10, "БИ-1803", 3)''') 

print(c.lastrowid)

c.execute('''INSERT INTO Groups VALUES (11, "БИ-1804", 3)''') 

print(c.lastrowid)




10
11


### **connection**
Этот доступный только для чтения атрибут обеспечивает соединение с базой данных SQLite, используемое объектом Cursor.
Пример использования:

In [ ]:
import sqlite3
conn = sqlite3.connect('Groups.db')
c = conn.cursor()

c.connection == conn

True

### **close()**
Закрытие курсора. С момента вызова курсор станет недоступен для использования

# **Задание**

Создайте базу данных в оперативной памяти. В ней сделайте таблицу с тремя строками. Добавьте хотя бы две записи в эту таблицу, а затем сделайте выборку с помощью 'Select *' (всех записей или любой конкретной по ключу) с выводом на экран.